<a href="https://colab.research.google.com/github/NicholasDelianedis/PIC16BProject/blob/main/PIC16B_Project%2BNumerical_Simplified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Entry

In [ ]:
pip install simfin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 5.5 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import simfin as sf
sf.set_api_key('H9OLpPRQ36sXqfhoOtyBb0Kwv4Dw0q3W')
sf.set_data_dir('~/simfin_data/')
quarterly = sf.load_income(variant='quarterly', market='us')
companies = sf.load_companies(market='us')
industries = sf.load_industries()
shares = sf.load_shareprices(variant='daily', market='us')
shares = shares.reset_index()
merge1 = pd.merge(quarterly, companies, how = 'left', on = 'SimFinId')
alldata = pd.merge(merge1, industries, how = 'left', on = 'IndustryId')
shares["NextDay"] = shares["Open"].shift(-1)
alldata["Date"] = alldata["Publish Date"]
shares2 = shares[["SimFinId", "Open", "Close", "Date", "NextDay"]]
alldata = pd.merge(alldata, shares2, how = "left", on = ["SimFinId", "Date"])
alldata["DeltaPrice"] = alldata.apply(lambda row: row["NextDay"] - row["Close"], axis=1)
alldata = alldata.drop(["Currency", "IndustryId"], axis = 1)
regdata = alldata[["SimFinId", "Shares (Basic)", "Revenue", "Cost of Revenue", "Gross Profit", "Operating Expenses", "Selling, General & Administrative", "Research & Development", "Depreciation & Amortization", "Operating Income (Loss)", "Non-Operating Income (Loss)", "Interest Expense, Net", "Pretax Income (Loss), Adj.", "Abnormal Gains (Losses)", "Pretax Income (Loss)", "Income Tax (Expense) Benefit, Net", "Income (Loss) from Continuing Operations", "Net Extraordinary Gains (Losses)", "Net Income", "Sector", "Industry", "Date", "Open", "Close", "NextDay", "DeltaPrice"]]
regdata = regdata.fillna(0)
regdata["NewDate"] = regdata["Date"]
for i in range(len(regdata["Date"])):
  n = (regdata["Date"][i].year - 2016) * 366
  n += (regdata["Date"][i].month - 1) * 31
  n += regdata["Date"][i].day
  regdata["NewDate"][i] = int(n)

Dataset "us-income-quarterly" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-companies" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "industries" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-shareprices-daily" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import train_test_split

newdata = regdata.drop(["Sector", "Industry", "Date", "NextDay"], axis = 1)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def lr(df = newdata):
  mlinx = []
  mliny = []
  regx = df.drop("DeltaPrice", axis = 1)
  for i in df["DeltaPrice"]:
    mliny.append(round(i, 2))
    mlinx.append([])
  for i in regx.columns:
    k = 0
    for j in regx[i]:
      mlinx[k].append(j)
      k += 1
  
  X2 = sm.add_constant(mlinx)
  est = sm.OLS(mliny, X2)
  est2 = est.fit()
  print(est2.summary())
  j = 1
  for i in regx.columns:
    print(str(j) + ": " + i)
    j += 1
  X_train, X_test, y_train, y_test = train_test_split(mlinx, mliny, test_size = 0.2, random_state = 42)
  LR = LinearRegression().fit(X_train, y_train)
  print("Test score:")
  print(LR.score(X_train, y_train))
  return X_train, y_train

In [ ]:
new = newdata.drop(["SimFinId", "Income Tax (Expense) Benefit, Net", "Income (Loss) from Continuing Operations", "Abnormal Gains (Losses)", "Pretax Income (Loss)"], axis = 1)
X_train, y_train = lr()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.136
Method:                 Least Squares   F-statistic:                     286.3
Date:                Wed, 08 Jun 2022   Prob (F-statistic):               0.00
Time:                        11:32:04   Log-Likelihood:            -1.8746e+05
No. Observations:               36180   AIC:                         3.750e+05
Df Residuals:                   36159   BIC:                         3.751e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4135      0.622     -0.665      0.5

In [ ]:
import tensorflow as tf
from tensorflow.keras import utils, datasets, layers, losses

In [ ]:
from numpy import array

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Normalization

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(22, 1)))
model.add(Dense(1400))
model.add(tf.keras.layers.Normalization(axis = 1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model1 = Sequential()
model1.add(Dense(22, input_dim=22, kernel_initializer='normal', activation='relu'))
model1.add(Dense(1))
model1.compile(optimizer='adam', loss='mse')

In [ ]:
def nn(sector="Technology", rnn = False):
  rnndf = regdata[regdata["Sector"] == sector]
  rnndf = rnndf.drop(["Sector", "Industry", "Date", "NextDay"], axis = 1)
  rnndf = rnndf[rnndf["Close"] != 0]
  rnndf["PctChg"] = rnndf.apply(lambda row: round(1000 * (row["DeltaPrice"] / row["Close"])) / 10, axis=1)
  rnndf = rnndf[rnndf["PctChg"] < 70]
  rnndf = rnndf[rnndf["PctChg"] > -70]
  rnnx = rnndf.drop(["DeltaPrice", "PctChg"], axis = 1)
  cmp = rnndf["SimFinId"]
  cmp = cmp.drop_duplicates()
  ryl = []
  rny = rnndf.sort_values("NewDate", axis = 0)
  for i in rny["PctChg"]:
    ryl.append(round(i, 2))
  rxl = []
  rnx = rnnx.sort_values("NewDate")
  for i in rnx["NewDate"]:
    rxl.append([])
  for i in rnx.columns:
    k = 0
    for j in rnx[i]:
      rxl[k].append(j)
      k += 1
  rxa = np.array(rxl)
  rya = np.array(ryl)
  if rnn:
    model.fit(rxa, rya, epochs=50, verbose=1)
  else:
    model1.fit(rxa, rya, epochs=500, verbose=1)

In [ ]:
nn(sector="Healthcare", rnn=False)

Epoch 1/500
187/187 [==============================] - 6s 4ms/step - loss: 4009573966938112.0000
Epoch 2/500
187/187 [==============================] - 1s 4ms/step - loss: 449215863980032.0000
Epoch 3/500
187/187 [==============================] - 1s 4ms/step - loss: 334298276167680.0000
Epoch 4/500
187/187 [==============================] - 1s 4ms/step - loss: 623538117541888.0000
Epoch 5/500
187/187 [==============================] - 1s 3ms/step - loss: 254735315632128.0000
Epoch 6/500
187/187 [==============================] - 1s 4ms/step - loss: 116395568267264.0000
Epoch 7/500
187/187 [==============================] - 1s 4ms/step - loss: 467193691111424.0000
Epoch 8/500
187/187 [==============================] - 1s 4ms/step - loss: 675180267438080.0000
Epoch 9/500
187/187 [==============================] - 1s 4ms/step - loss: 482984104820736.0000
Epoch 10/500
187/187 [==============================] - 1s 4ms/step - loss: 120218886078464.0000
Epoch 11/500
187/187 [================

In [ ]:
nn(sector='Healthcare', rnn=True)

Epoch 1/50
187/187 [==============================] - 8s 34ms/step - loss: 230884775034880.0000
Epoch 2/50
187/187 [==============================] - 6s 35ms/step - loss: 68809075458048.0000
Epoch 3/50
187/187 [==============================] - 6s 33ms/step - loss: 21307706048512.0000
Epoch 4/50
187/187 [==============================] - 6s 33ms/step - loss: 9387806556160.0000
Epoch 5/50
187/187 [==============================] - 6s 33ms/step - loss: 8657564073984.0000
Epoch 6/50
187/187 [==============================] - 6s 34ms/step - loss: 2411129470976.0000
Epoch 7/50
187/187 [==============================] - 6s 33ms/step - loss: 1299764609024.0000
Epoch 8/50
187/187 [==============================] - 6s 33ms/step - loss: 1365610463232.0000
Epoch 9/50
187/187 [==============================] - 6s 33ms/step - loss: 798963204096.0000
Epoch 10/50
187/187 [==============================] - 6s 33ms/step - loss: 606213898240.0000
Epoch 11/50
187/187 [==============================] - 6s

In [ ]:
from matplotlib import pyplot as plt 
plt.scatter(X_train[: , 0], y_train, 'x')
plt.show()

TypeError: ignored

In [ ]:
X_train[20000]